**Names** Safyre Anderson, Howard Wen , Vamsi Sakhamuri

**Emails** safyre@berkeley.edu, howard.wen1@gmail.com, vamsi@ischool.berkeley.edu 

**Time of Initial Submission:** February 18th, 2016 8am PST

**Section** W261-3, Spring 2016  

**Week** 5 Homework

## HW 5.0
*What is a data warehouse? What is a Star schema? When is it used?*

## HW 5.1
*In the database world What is 3NF? Does machine learning use data in 3NF? If so why?  
In what form does ML consume data?  
Why would one use log files that are denormalized?  *

## HW 5.2
*Using MRJob, implement a hashside join (memory-backed map-side) for left,right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)*

*Justify which table you chose as the Left table in this hashside join.*

*Please report the number of rows resulting from:*

*(1) Left joining Table Left with Table Right  
(2) Right joining Table Left with Table Right  
(3) Inner joining Table Left with Table Right*  

In [1]:
%%writefile msft_preprocess.py

#!/usr/bin/python
import sys
import re

filename_r = sys.argv[1]    #original file
filename_w = sys.argv[2]    #modified file

start_C = 0
start_V = 0

with open(filename_r,"r") as fr:
    for line_r in fr:
        line_rs = []
        line_rs = [subs.strip('\"') for subs in re.split(',',line_r.strip())]
        if((start_V==1) and (start_C==1) and line_rs[0]=='V'):    #second and beyond vroot's of a customer
            line_rs.append('C')
            line_rs.append(case_ID)
            with open(filename_w,"a+") as fw:
                fw.write(",".join(line_rs)+"\n")
        elif((start_V==0) and (start_C==1) and line_rs[0] =='V'): #first Vroot of a customer
            line_rs.append('C')
            line_rs.append(case_ID)
            with open(filename_w,"a+") as fw:        
                fw.write(",".join(line_rs)+"\n")
                start_V = 1
        elif((start_V==1) and (start_C==1) and line_rs[0] =='C'): 
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1
        elif((start_V==0) and (start_C==1) and line_rs[0] =='C'): 
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1
        elif(start_V==0 and (start_C==0) and (line_rs[0]=='C')): 
            case_ID = line_rs[1]
            start_V = 0
            start_C = 1

Writing msft_preprocess.py


In [7]:
!head anonymous-msweb.data

I,4,"www.microsoft.com","created by getlog.pl"
T,1,"VRoot",0,0,"VRoot"
N,0,"0"
N,1,"1"
T,2,"Hide1",0,0,"Hide"
N,0,"0"
N,1,"1"
A,1287,1,"International AutoRoute","/autoroute"
A,1288,1,"library","/library"
A,1289,1,"Master Chef Product Information","/masterchef"


In [8]:
!tail anonymous-msweb.data

V,1041,1
C,"42709",42709
V,1001,1
V,1003,1
C,"42710",42710
V,1035,1
V,1001,1
V,1018,1
C,"42711",42711
V,1008,1


In [2]:
!rm anonymous-msweb.data_mod
!python msft_preprocess.py anonymous-msweb.data anonymous-msweb.data_mod

rm: anonymous-msweb.data_mod: No such file or directory


In [3]:
!echo "\nFirst 10 lines of the post-processed file\n"
!head anonymous-msweb.data_mod
!echo "\nLast 10 lines of the post-processed file\n"
!tail anonymous-msweb.data_mod


First 10 lines of the post-processed file

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005

Last 10 lines of the post-processed file

V,1123,1,C,42708
V,1038,1,C,42708
V,1026,1,C,42708
V,1041,1,C,42708
V,1001,1,C,42709
V,1003,1,C,42709
V,1035,1,C,42710
V,1001,1,C,42710
V,1018,1,C,42710
V,1008,1,C,42711


In [17]:
%%writefile msft_preprocess1.py

#!/usr/bin/python
import sys
import re

filename_r = sys.argv[1]
filename_w = sys.argv[2]

with open(filename_r,"r") as fr:
    for line_r in fr:
        line_rs = [subs.strip('\"') for subs in re.split(',',line_r.strip())]
        if(line_rs[0]=='A'):
            with open(filename_w,"a+") as fw:
                fw.write(",".join(line_rs) + '\n')

Overwriting msft_preprocess1.py


In [18]:
!rm anonymous-msweb.data_attr
!python msft_preprocess1.py anonymous-msweb.data anonymous-msweb.data_attr

In [19]:
!echo "\nFirst 10 lines of the post-processed file\n"
!head anonymous-msweb.data_attr
!echo "\nLast 10 lines of the post-processed file\n"
!tail anonymous-msweb.data_attr


First 10 lines of the post-processed file

A,1287,1,International AutoRoute,/autoroute
A,1288,1,library,/library
A,1289,1,Master Chef Product Information,/masterchef
A,1297,1,Central America,/centroam
A,1215,1,For Developers Only Info,/developer
A,1279,1,Multimedia Golf,/msgolf
A,1239,1,Microsoft Consulting,/msconsult
A,1282,1,home,/home
A,1251,1,Reference Support,/referencesupport
A,1121,1,Microsoft Magazine,/magazine

Last 10 lines of the post-processed file

A,1265,1,Source Safe Support,/ssafesupport
A,1271,1,mdsn,/mdsn
A,1129,1,ActiveX Data Objects,/ado
A,1018,1,isapi,/isapi
A,1193,1,Office Developer Support,/offdevsupport
A,1219,1,Corporate Advertising Content,/ads
A,1030,1,Windows NT Server,/ntserver
A,1182,1,Fortran,/fortran
A,1100,1,MS in Education,/education
A,1210,1,SNA Support,/snasupport


In [56]:
%%writefile reducersideinnerjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class innerjoin(MRJob):
    def mapper(self, _, line):
        x = line.strip().split(",")
        if ((len(x) == 5) and (x[0]=='V')):
            yield x[1], ("lefttable", x[0], x[2], x[3],x[4])
        elif ((len(x) ==5) and (x[0]=='A')):
            yield x[1], ("righttable", x[0],x[2],x[3],x[4])   

    def reducer(self, key, values):
        vroots = []
        web = []
        for val in values:
            if val[0] == 'lefttable':
                vroots.append(val)
            else:
                web.append(val)
        for v in vroots:
            for w in web:
                yield None, [key] + v[1:] + w[1:]

if __name__ == '__main__':
    innerjoin.run()

Overwriting reducersideinnerjoin.py


In [63]:
%%writefile reducersideleftjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class leftjoin(MRJob):
    def mapper(self, _, line):
        x = line.strip().split(",")
        if ((len(x) == 5) and (x[0]=='V')):
            yield x[1], ("lefttable", x[0], x[2], x[3],x[4])
        elif ((len(x) ==5) and (x[0]=='A')):
            yield x[1], ("righttable", x[0],x[2],x[3],x[4])   

    def reducer(self, key, values):
        vroots = []
        web = []
        for val in values:
            if val[0] == 'lefttable':
                vroots.append(val)
            else:
                web.append(val)
        for v in vroots:
            if(len(web)==0):
                yield None,[key] + v[1:]
            else:
                for w in web:
                    yield None, [key] + v[1:] + w[1:]

if __name__ == '__main__':
    leftjoin.run()

Overwriting reducersideleftjoin.py


In [67]:
%%writefile reducersiderightjoin.py
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from mrjob.compat import get_jobconf_value
 
class rightjoin(MRJob):
    def mapper(self, _, line):
        x = line.strip().split(",")
        if ((len(x) == 5) and (x[0]=='V')):
            yield x[1], ("lefttable", x[0], x[2], x[3],x[4])
        elif ((len(x) ==5) and (x[0]=='A')):
            yield x[1], ("righttable", x[0],x[2],x[3],x[4])   

    def reducer(self, key, values):
        vroots = []
        web = []
        for val in values:
            if val[0] == 'lefttable':
                vroots.append(val)
            else:
                web.append(val)
        for w in web:
            if(len(vroots)==0):t
                yield None,[key] + w[1:]
            else:
                for v in vroots:
                    yield None, [key] + w[1:] + v[1:]

if __name__ == '__main__':
    rightjoin.run()

Overwriting reducersiderightjoin.py


In [65]:
%reload_ext autoreload
%autoreload 2

from reducersideinnerjoin import innerjoin

mr_job = innerjoin(args=['anonymous-msweb.data_mod','anonymous-msweb.data_attr'])

with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        count = count + 1
print "\n"
print "There are %s records" %count



There are 98654 records


In [66]:
%reload_ext autoreload
%autoreload 2

from reducersideleftjoin import leftjoin

mr_job = leftjoin(args=['anonymous-msweb.data_mod','anonymous-msweb.data_attr'])

with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        count = count + 1
print "\n"
print "There are %s records" %count



There are 98654 records


In [68]:
%reload_ext autoreload
%autoreload 2

from reducersiderightjoin import rightjoin

mr_job = rightjoin(args=['anonymous-msweb.data_mod','anonymous-msweb.data_attr'])

with mr_job.make_runner() as runner: 
    runner.run()
    count = 0
    # stream_output: get access of the output 
    for line in runner.stream_output():
        key,value =  mr_job.parse_output_line(line)
        count = count + 1
print "\n"
print "There are %s records" %count



There are 98663 records
